**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
!pip install scikit-video

import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

import torch
import torch.nn as nn
import torch.nn.functional as F 
from torch import optim

     |████████████████████████████████| 2.3MB 2.8MB/s 


# MiniProject on Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [0]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [0]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self,e):
        self.epsilon = e

    def act(self,s,train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self,s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The function act enables our agent to take an action according to what state we are in. The epsilon is essential because it enables the algorithm to explore its environment by sometimes choosing a random action instead of a leared action.

Indeed, in Reinforcement Learning algorithms we have a trade-off between exploitation (choose a learned action to maximize its reward) and exploration (choose a random action to explore another area of the its environment).

The epsilon enables to set this trade-off by taking a random action with an epsilon probability at each state.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [0]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate((self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate((
                               self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [0]:
# parameters
size = 13
T=200
temperature=0.3
epochs_train=15 # set small when debugging
epochs_test=10 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The arrays position seems to entirely code the position of the rat but also the feasibility of its action. Indeed, when the rat is nearby a wall and it makes an action that decides to go toward the wall the arrays position indicates that it is not possible and the rat will bounce on it.

The array board encode the rewards the rat can get, whenever is getting to a new state having cheese or poison, he wil get a reward of 1 or -1, and this reward will vanish (become equal to 0 as the rat consumed it). This is the role of the board array.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [0]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        return np.random.randint(self.n_action)

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [0]:
def test(agent,env,epochs,prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        
        ##### FILL IN HERE
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will end
        game_over = False
    
        win = 0
        lose = 0
    
        while not game_over:
            # The agent performs an action
            action = agent.act(state)
    
            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)
    
            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
        
        # Save as a mp4
        env.draw(prefix+str(e))
        #print('saved as '+prefix+str(e)+'.mp4' )

        # Update stats
        score = score + win-lose

        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score/(1+e)))
    print('Final score: '+str(score/epochs))

In [8]:
# Initialize the game
env = Environment(grid_size=size, max_time=T,temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env,epochs_test,prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 6.5/10.0. Average score (-3.5)
Win/lose count 12.5/12.0. Average score (-1.5)
Win/lose count 11.5/13.0. Average score (-1.5)
Win/lose count 12.0/21.0. Average score (-3.375)
Win/lose count 8.0/8.0. Average score (-2.7)
Win/lose count 12.0/10.0. Average score (-1.9166666666666667)
Win/lose count 9.0/12.0. Average score (-2.0714285714285716)
Win/lose count 9.0/10.0. Average score (-1.9375)
Win/lose count 11.5/8.0. Average score (-1.3333333333333333)
Win/lose count 12.5/14.0. Average score (-1.35)
Final score: -1.35


***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\le T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

We have : 
\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\gamma^0 r(s_0, a_0) + \sum_{1 \le t \le T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Let us do a change of variable :
\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[r(s_0, a_0) + \gamma \sum_{0 \le t \le T-1}\gamma^{t}r(s_{t},a_{t})|p_\pi(s_{0}),p_\pi(a_{0})] \> .
\end{equation*}

So :
\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

$\pi^*$ is optimal if and only if $\forall a', Q^{\pi^*}(s',a') = \max_{a'}{Q^*(s',a')}$ ie each action taken by theoptimal policy paves the way to the best reward possible.

We want our policy to be as close as possible to the optimal one. Minimizing the MSE si a way to do it.


***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [0]:
class Memory(object):
    def __init__(self, max_memory=100):
        self.max_memory = max_memory
        self.memory = list()

    def remember(self, m):
        self.memory.append(m)
        if len(self.memory) > self.max_memory :
            del self.memory[0]

    def random_access(self):
        return self.memory[np.random.randint(len(self.memory))]

***
The pipeline we will use for training is given below:

In [0]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)

            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        #agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [0]:
class DQN(Agent):
    def __init__(self, grid_size,  epsilon = 0.1, memory_size=100, batch_size = 16, n_state=2):
        super(DQN, self).__init__(epsilon = epsilon)

        # Discount for Q learning
        self.discount = 0.99
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return self.model(torch.tensor(s)[None]).argmax()

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember( [torch.tensor(s_), torch.tensor(n_s_), a_, r_, game_over_] )
        
        input_states = torch.zeros((self.batch_size, 5,5,self.n_state))
        target_q = torch.zeros((self.batch_size, 4))
        
        for i in range(self.batch_size):
            input_i = self.memory.random_access()

            if input_i[-1]:
                input_states[i] = input_i[0]
                target_q[i, :] = self.model(input_i[0][None])
            else:
                input_states[i] = input_i[0]
                target_q[i,:] = self.model(input_i[0][None])
                target_q[i, input_i[2]] = input_i[3] + self.discount * self.model(input_i[1][None]).max()

        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q[target_q<-3] = -3
        target_q[target_q>3] = 3

        l = self.model.train_on_batch(input_states, target_q)
        
        # Update
        l.backward()
        self.optim.step()
        self.optim.zero_grad()
        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model
            
class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        ####### FILL IN
        model = Model_FC()
        self.optim = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)#, momentum=0.0)
        self.model = model


class Model_FC(nn.Module):
    def __init__(self, n_hidden=64, n_hidden_2=16, states_flatten_size = 50):
        super(Model_FC, self).__init__()
        self.fc1 = nn.Linear(states_flatten_size, n_hidden)
        self.fc2 = nn.Linear(n_hidden, n_hidden_2)
        self.fc3 = nn.Linear(n_hidden_2, 4)
    
    def forward(self, x):
        x = x.view(x.size(0), -1).float()
        x = F.relu(self.fc1(x))
        x = F.leaky_relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def train_on_batch(self, input_states, target_q) :
        L = torch.nn.MSELoss()(target_q, self.forward(input_states))
        return L
        

In [12]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent_fc = DQN_FC(size, lr=.001, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent_fc, env, epochs_train, prefix='fc_train')
test(agent_fc,env,epochs_test, prefix='fc_test')
HTML(display_videos('fc_test9.mp4'))

Epoch 000/015 | Loss 0.0022 | Win/lose count 3.5/5.0 (-1.5)
Epoch 001/015 | Loss 0.0038 | Win/lose count 4.0/3.0 (1.0)
Epoch 002/015 | Loss 0.0005 | Win/lose count 3.0/7.0 (-4.0)
Epoch 003/015 | Loss 0.0019 | Win/lose count 3.0/0 (3.0)
Epoch 004/015 | Loss 0.0012 | Win/lose count 4.5/2.0 (2.5)
Epoch 005/015 | Loss 0.0047 | Win/lose count 4.0/2.0 (2.0)
Epoch 006/015 | Loss 0.0005 | Win/lose count 8.0/6.0 (2.0)
Epoch 007/015 | Loss 0.0033 | Win/lose count 3.0/4.0 (-1.0)
Epoch 008/015 | Loss 0.0014 | Win/lose count 6.0/4.0 (2.0)
Epoch 009/015 | Loss 0.0008 | Win/lose count 5.5/3.0 (2.5)
Epoch 010/015 | Loss 0.0024 | Win/lose count 3.5/4.0 (-0.5)
Epoch 011/015 | Loss 0.0057 | Win/lose count 4.5/0 (4.5)
Epoch 012/015 | Loss 0.0029 | Win/lose count 7.5/1.0 (6.5)
Epoch 013/015 | Loss 0.0006 | Win/lose count 5.0/3.0 (2.0)
Epoch 014/015 | Loss 0.0035 | Win/lose count 5.0/3.0 (2.0)
Win/lose count 3.5/1.0. Average score (2.5)
Win/lose count 14.0/5.0. Average score (5.75)
Win/lose count 6.0/1.0. A

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [0]:
class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        ###### FILL IN
        model = Model_CNN(self.n_state)
        self.optim = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)#, momentum=0.0)
        self.model = model

class Model_CNN(nn.Module):
    def __init__(self, n_state):
        super(Model_CNN, self).__init__()
        self.conv1 = nn.Conv2d(n_state, 16, 3)
        self.conv2 = nn.Conv2d(16, 64, 3)
        self.fc3 = nn.Linear(64, 4)
    
    def forward(self, x):
        x = x.view(x.size(0), x.size(3), x.size(1), x.size(2)).float()
        x = F.relu(self.conv1(x))
        x = F.leaky_relu(self.conv2(x))
        x = x.view(x.size(0), -1)
        x = self.fc3(x)
        return x

    def train_on_batch(self, input_states, target_q) :
        L = torch.nn.MSELoss()(target_q, self.forward(input_states))
        return L

In [14]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent_cnn = DQN_CNN(size, lr=.001, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent_cnn,env,epochs_train,prefix='cnn_train')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
HTML(display_videos('cnn_test0.mp4'))

Epoch 000/015 | Loss 0.0022 | Win/lose count 0.5/9.0 (-8.5)
Epoch 001/015 | Loss 0.0001 | Win/lose count 0.5/2.0 (-1.5)
Epoch 002/015 | Loss 0.0019 | Win/lose count 4.0/3.0 (1.0)
Epoch 003/015 | Loss 0.0037 | Win/lose count 4.5/2.0 (2.5)
Epoch 004/015 | Loss 0.0016 | Win/lose count 6.0/3.0 (3.0)
Epoch 005/015 | Loss 0.0001 | Win/lose count 2.0/0 (2.0)
Epoch 006/015 | Loss 0.0023 | Win/lose count 3.5/0 (3.5)
Epoch 007/015 | Loss 0.0014 | Win/lose count 7.5/2.0 (5.5)
Epoch 008/015 | Loss 0.0007 | Win/lose count 4.5/2.0 (2.5)
Epoch 009/015 | Loss 0.0010 | Win/lose count 7.0/3.0 (4.0)
Epoch 010/015 | Loss 0.0030 | Win/lose count 9.5/7.0 (2.5)
Epoch 011/015 | Loss 0.0004 | Win/lose count 1.0/0 (1.0)
Epoch 012/015 | Loss 0.0026 | Win/lose count 5.5/2.0 (3.5)
Epoch 013/015 | Loss 0.0035 | Win/lose count 8.0/4.0 (4.0)
Epoch 014/015 | Loss 0.0030 | Win/lose count 13.5/0 (13.5)
Win/lose count 15.5/3.0. Average score (12.5)
Win/lose count 7.5/2.0. Average score (9.0)
Win/lose count 4.5/2.0. Avera

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [15]:
env = Environment(grid_size=size, max_time=T,temperature=0.3)
'''
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
'''
print('Test of the CNN')
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 14.5/2.0. Average score (12.5)
Win/lose count 6.0/1.0. Average score (8.75)
Win/lose count 12.5/1.0. Average score (9.666666666666666)
Win/lose count 11.5/5.0. Average score (8.875)
Win/lose count 4.5/1.0. Average score (7.8)
Win/lose count 2.5/1.0. Average score (6.75)
Win/lose count 11.0/2.0. Average score (7.071428571428571)
Win/lose count 9.0/3.0. Average score (6.9375)
Win/lose count 8.0/3.0. Average score (6.722222222222222)
Win/lose count 13.5/1.0. Average score (7.3)
Final score: 7.3
Test of the FC
Win/lose count 4.5/3.0. Average score (1.5)
Win/lose count 7.0/3.0. Average score (2.75)
Win/lose count 3.5/1.0. Average score (2.6666666666666665)
Win/lose count 5.5/3.0. Average score (2.625)
Win/lose count 10.0/3.0. Average score (3.5)
Win/lose count 11.5/1.0. Average score (4.666666666666667)
Win/lose count 8.0/2.0. Average score (4.857142857142857)
Win/lose count 11.0/1.0. Average score (5.5)
Win/lose count 5.5/3.0. Average score (5.166666666666667

In [16]:
HTML(display_videos('cnn_test9.mp4'))

In [17]:
HTML(display_videos('fc_test9.mp4'))

It seems that the agent does not explore a lot as it does not want to get bad rewards. It may stay in the same area of the map to limit its 'loss', it bad rewards.
We should add a dynamic exploration feature for this agent in order to let him get good rewards on the whole map whie still avoiding negative rewards.


***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [0]:
def train_explore(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0
    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False
        win = 0
        lose = 0
        while not game_over:
            # The agent performs an action
            action = agent.act(state)
            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action, train=True)
            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward
            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)
        # Save as a mp4
        env.draw(prefix+str(e))
        # Update stats
        score += win-lose
        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        #agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
        
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature
        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size))
        # coordinate of the cat
        self.x = 0
        self.y = 1
        # self time
        self.t = 0
        self.scale=16
        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))

    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)
        self.to_draw[t,:,:,:]=b

    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""
        self.get_frame(int(self.t))
        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward=0
        if train:
            reward = -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1
        reward = reward + self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        #############################################################################
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        ###############################################################################
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]
        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""
        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]

        bonus = 0.5*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size,self.grid_size)

        malus = -1.0*np.random.binomial(1,self.temperature,size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))
        malus[bonus>0]=0
        self.board = bonus + malus
        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0
        state = np.concatenate((self.malus_position.reshape(self.grid_size, self.grid_size,1),
                                self.board.reshape(self.grid_size, self.grid_size,1),
                        self.position.reshape(self.grid_size, self.grid_size,1)),axis=2)
        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state


In [19]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.001, epsilon = 0.1, memory_size=2000, batch_size = 32, n_state=3)
train_explore(agent, env, 50, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore49.mp4'))

Epoch 000/050 | Loss 0.0024 | Win/lose count 2.0/20.70000000000003 (-18.70000000000003)
Epoch 001/050 | Loss 0.0043 | Win/lose count 3.4/26.900000000000112 (-23.500000000000114)
Epoch 002/050 | Loss 0.0043 | Win/lose count 5.5/20.600000000000026 (-15.100000000000026)
Epoch 003/050 | Loss 0.0037 | Win/lose count 1.7999999999999998/23.40000000000006 (-21.60000000000006)
Epoch 004/050 | Loss 0.0034 | Win/lose count 3.5/22.60000000000006 (-19.10000000000006)
Epoch 005/050 | Loss 0.0017 | Win/lose count 2.5/20.900000000000034 (-18.400000000000034)
Epoch 006/050 | Loss 0.0028 | Win/lose count 3.5/19.900000000000016 (-16.400000000000016)
Epoch 007/050 | Loss 0.0026 | Win/lose count 8.000000000000002/20.70000000000003 (-12.70000000000003)
Epoch 008/050 | Loss 0.0018 | Win/lose count 3.0/21.100000000000033 (-18.100000000000033)
Epoch 009/050 | Loss 0.0018 | Win/lose count 7.000000000000002/17.999999999999986 (-10.999999999999984)
Epoch 010/050 | Loss 0.0017 | Win/lose count 9.700000000000001/21

In [20]:
# Evaluation
test(agent,env,epochs_test,prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore9.mp4'))

Win/lose count 15.5/2.0. Average score (13.5)
Win/lose count 12.5/2.0. Average score (12.0)
Win/lose count 11.5/0. Average score (11.833333333333334)
Win/lose count 12.0/1.0. Average score (11.625)
Win/lose count 14.5/0. Average score (12.2)
Win/lose count 5.0/1.0. Average score (10.833333333333334)
Win/lose count 4.5/0. Average score (9.928571428571429)
Win/lose count 7.5/2.0. Average score (9.375)
Win/lose count 7.0/1.0. Average score (9.0)
Win/lose count 17.5/1.0. Average score (9.75)
Final score: 9.75


***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***